In [77]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model

In [78]:
print(tf.test.is_gpu_available())

False


In [79]:
import requests
# Download and preprocess the data from the URL
#url = "https://www.gutenberg.org/files/55/55-0.txt"
url = "https://drive.google.com/file/d/19vcLOfWf3fkHTXAq8VEvNbtiwVOCryhR/view?usp=sharing"

response = requests.get(url)
wizard_of_oz = response.text


In [80]:
start_text = "*** START OF THE PROJECT GUTENBERG EBOOK THE WONDERFUL WIZARD OF OZ ***"
# Find the index where the start_text occurs
start_index = wizard_of_oz.find(start_text) + 28

# Extract the text after the start_text
wizard_of_oz = wizard_of_oz[start_index + len(start_text):]
print(wizard_of_oz[:250] + "...")

ible" content="IE=edge;"><style nonce="3T0VVUSMNjUJpJnOG92yXg">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';...


In [81]:
end_text = "*** END OF THE PROJECT GUTENBERG EBOOK THE WONDERFUL WIZARD OF OZ ***"

# Find the index where the end_text occurs
end_index = wizard_of_oz.find(end_text)

# Extract the text before the end_text
wizard_of_oz = wizard_of_oz[:end_index]

# show the final of the text
print(wizard_of_oz[-200:])

" src="https://www.gstatic.com/_/apps-fileview/_/js/k=apps-fileview.v.en_GB.1pe6cfMhEoU.O/am=gAE/d=1/rs=AO0039vbZV5ERKDi0SMGC1bCI1bc5DhF5A/m=v,wb" nonce="jlDD5qJPYxK-ZDlKdUeHmw"></script></body></html


In [82]:
# Preprocess the data
sentences = wizard_of_oz.split(".")
sentences = [s.strip() for s in sentences if len(s) > 0]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1


In [83]:
# Create input-output sequences
input_sequences = []
for sentence in sentences:
    seq = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(seq)):
        n_gram_seq = seq[:i+1]
        input_sequences.append(n_gram_seq)

In [84]:
# Pad sequences
max_seq_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')

In [85]:
# Split input-output sequences into input (X) and output (y)
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [86]:
# Convert y to categorical labels
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [87]:

model= tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])
model.build()
print("model built")

model built


In [88]:
# Plot the model architecture
#plot_model(model, show_shapes=True, show_layer_names=True)


In [89]:
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_20                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_21                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_22                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_23                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [90]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [91]:
# Train the model
#model.fit(X, y, epochs=100, verbose=1)

model.fit(X, y, epochs=1, verbose=1)

 10/428 ━━━━━━━━━━━━━━━━━━━━ 7:21 1s/step - accuracy: 0.0457 - loss: 7.3185

KeyboardInterrupt: 

In [ ]:
# Function to generate sentence completion
def complete_sentence(model, tokenizer, initial_text, max_length=20):
    input_text = initial_text
    for _ in range(max_length):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        encoded_text = pad_sequences([encoded_text], maxlen=max_seq_len-1, padding='pre')
        predicted_probabilities = model.predict(encoded_text, verbose=0)[0]
        predicted_index = tf.argmax(predicted_probabilities, axis=-1).numpy()
        predicted_word = tokenizer.index_word[predicted_index]
        input_text += " " + predicted_word
        if predicted_word == '.':
            break
    return input_text + "..."

In [ ]:
# # Generate a completed sentence
# completed_sentence = complete_sentence(model, tokenizer, "Dorothy lived in", max_length=10)
# print(completed_sentence)
# # Generate a completed sentence
# completed_sentence = complete_sentence(model, tokenizer, "While she stood looking", max_length=10)
# print(completed_sentence)
# # Generate a completed sentence
# completed_sentence = complete_sentence(model, tokenizer, "The little old woman took the slate from her nose", max_length=10)
# print(completed_sentence)

Dorothy lived in 0 0 0 0 0 0 0 0 0 0...


In [ ]:
completed_sentence = complete_sentence(model, tokenizer, "Define features in images.", max_length=10)
print(completed_sentence)

While she stood looking 0 0 0 0 0 0 0 0 0 0...


The little old woman took the slate from her nose box docs docs docs docs docs docs docs docs docs...
